In [1]:
from typing import Dict,Callable

from tempfile import gettempdir
import matplotlib.pyplot as plt
import numpy as np

import torch
from torch import optim,Tensor,unsqueeze
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import torch.autograd
import torch.nn as nn
from torchvision.models.resnet import resnet50,resnet18
import torchvision.transforms as transforms
from torchvision import datasets
from torchvision.utils import save_image
import torch.nn.functional as F

from newdataset import MyTrainDataset, my_dataset_worker_init_func

from tqdm import tqdm

from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import AgentDataset, EgoDataset
from l5kit.rasterization import build_rasterizer
from l5kit.evaluation import write_pred_csv, compute_metrics_csv, write_gt_csv, read_gt_csv
from l5kit.evaluation.chop_dataset import MIN_FUTURE_STEPS
from metrics import neg_multi_log_likelihood, time_displace, average_displacement_error_oracle, average_displacement_error_mean, final_displacement_error_oracle, final_displacement_error_mean
from l5kit.geometry import transform_points
from l5kit.visualization import PREDICTED_POINTS_COLOR, TARGET_POINTS_COLOR, draw_trajectory
from prettytable import PrettyTable
from pathlib import Path

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="True"

D:\Anaconda3\envs\lgsvl\lib\site-packages\l5kit\dataset\select_agents.py:32: UserWarning: Windows detected. BLOSC_NOLOCK has not been set as it causes memory leaks on Windows.However, writing the mask with this config may be inconsistent.
  "Windows detected. BLOSC_NOLOCK has not been set as it causes memory leaks on Windows."


# 数据集构建

In [2]:
# set env variable for data
os.environ["L5KIT_DATA_FOLDER"] = "E:/Downloads/lyft-motion-prediction-autonomous-vehicles"
dm = LocalDataManager(None)
# get config
cfg = load_config_data("./control_config.yaml")
print(cfg)

{'format_version': 4, 'mode': {'load_mode': False}, 'model_params': {'history_step_size': 1, 'history_num_frames': 2, 'future_step_size': 1, 'future_num_frames': 50, 'step_time': 0.1, 'render_ego_history': True}, 'raster_params': {'raster_mode': 0, 'raster_size': [240, 240], 'pixel_size': [0.5, 0.5], 'ego_center': [0.25, 0.5], 'map_type': 'py_semantic', 'satellite_map_key': 'aerial_map/aerial_map.png', 'semantic_map_key': 'semantic_map/semantic_map.pb', 'dataset_meta_key': 'meta.json', 'filter_agents_threshold': 0.5, 'disable_traffic_light_faces': False, 'set_origin_to_bottom': True}, 'train_data_loader': {'key': 'scenes/sample.zarr', 'batch_size': 8, 'shuffle': True, 'num_workers': 2}, 'val_data_loader': {'key': 'scenes/sample.zarr', 'batch_size': 8, 'shuffle': False, 'num_workers': 2}, 'scale': 1, 'train_params': {'device': 1, 'epochs': 1}}


In [3]:
eval_cfg = cfg["val_data_loader"]
rasterizer = build_rasterizer(cfg, dm)
eval_zarr = ChunkedDataset(dm.require(eval_cfg["key"])).open()
eval_dataset = AgentDataset(cfg, eval_zarr, rasterizer)
print(len(eval_dataset))
print(eval_dataset[0].keys())
# print(len(eval_dataset))

eval_dataset = MyTrainDataset('eval', cfg, dm, len(eval_dataset),raster_mode = cfg["raster_params"]["raster_mode"])
eval_dataloader = DataLoader(
    eval_dataset,
    shuffle=eval_cfg["shuffle"], 
    batch_size=eval_cfg["batch_size"],
    num_workers=eval_cfg["num_workers"],
    prefetch_factor = 2,
    pin_memory = True,
    worker_init_fn=my_dataset_worker_init_func
)
cv_path = "E:/Downloads/lyft-motion-prediction-autonomous-vehicles/cv.csv"
po_path = "E:/Downloads/lyft-motion-prediction-autonomous-vehicles/po.csv"
eval_gt_path = "E:/Downloads/lyft-motion-prediction-autonomous-vehicles/gt.csv"

111634
dict_keys(['frame_index', 'image', 'target_positions', 'target_yaws', 'target_velocities', 'target_availabilities', 'history_positions', 'history_yaws', 'history_velocities', 'history_availabilities', 'world_to_image', 'raster_from_agent', 'raster_from_world', 'agent_from_world', 'world_from_agent', 'centroid', 'yaw', 'extent', 'history_extents', 'future_extents', 'curr_speed', 'scene_index', 'host_id', 'timestamp', 'track_id'])


# 模型构建

## 参数

In [4]:
# 基本参数
if cfg["train_params"]["device"]:
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.enabled = True 
    torch.backends.cudnn.benchmark = True
else:
    device = torch.device("cpu")

## 模型

In [5]:
from physics import ConstantVelocityHeading, PhysicsOracle

cv_model = ConstantVelocityHeading(sec_from_now=5)
physics_oracle = PhysicsOracle(sec_from_now=5)

# 模型评估

In [6]:
# ==== EVAL LOOP
# store information for evaluation
future_cv_offsets_pd = []
future_po_offsets_pd = []
gt_coords_offsets_pd = []
timestamps = []
agent_ids = []
availability = []
confs = []
tr_it = iter(eval_dataloader)
progress_bar = tqdm(range(len(eval_dataloader)//cfg['scale']),position=0)

for i in progress_bar:
    try:
        data,_ = next(tr_it)
    except StopIteration:
        tr_it = iter(eval_dataloader)
        data,_ = next(tr_it)
    y_cv, confidences = cv_model(data)
    y_po, _ = physics_oracle(data)
    # convert agent coordinates into world offsets
    cv_coords = y_cv.detach().cpu().numpy()
    po_coords = y_po.detach().cpu().numpy()
    gt_coords = data['target_positions'].numpy()
    world_from_agents = data['world_from_agent'].numpy()
    centroids = data["centroid"].numpy()
    coords_off = []
    coords_cv.append(transform_points(cv_coords, world_from_agents) - centroids[:, None, :2])
    coords_po.append(transform_points(po_coords, world_from_agents) - centroids[:, None, :2])
    cv_offset = np.stack([coords_offseti for coords_offseti in coords_cv],1)
    po_offset = np.stack([coords_offseti for coords_offseti in coords_po],1)
    gt_offset = transform_points(gt_coords, world_from_agents) - centroids[:, None, :2]
    
    future_cv_offsets_pd.append(np.stack(cv_offset))
    future_po_offsets_pd.append(np.stack(po_offset))
    gt_coords_offsets_pd.append(np.stack(gt_offset))
    timestamps.append(data["timestamp"].numpy().copy())
    agent_ids.append(data["track_id"].numpy().copy())
    availability.append(data["target_availabilities"].numpy().copy())
    confs.append(confidences.detach().cpu().numpy().copy())

  0%|                                                                                        | 0/13955 [00:05<?, ?it/s]


RuntimeError: DataLoader worker (pid(s) 3640, 9264) exited unexpectedly

In [ ]:
write_pred_csv(cv_path,
               timestamps=np.concatenate(timestamps),
               track_ids=np.concatenate(agent_ids),
               coords=np.concatenate(future_coords_offsets_pd),
               confs=np.concatenate(confs)
              )
write_pred_csv(po_path,
               timestamps=np.concatenate(timestamps),
               track_ids=np.concatenate(agent_ids),
               coords=np.concatenate(future_coords_offsets_pd),
               confs=np.concatenate(confs)
              )
write_gt_csv(eval_gt_path,timestamps=np.concatenate(timestamps),
               track_ids=np.concatenate(agent_ids),
               coords=np.concatenate(gt_coords_offsets_pd),avails=np.concatenate(availability))

In [ ]:
metrics = compute_metrics_csv(eval_gt_path, pred_path, [
                              final_displacement_error_oracle, final_displacement_error_mean, average_displacement_error_oracle, average_displacement_error_mean, time_displace])
for metric_name, metric_mean in metrics.items():
    print(metric_name, metric_mean)
    if metric_name=="time_displace":
        FDE = metric_mean
print('FDE1s: {}, FDE3s: {}, FDE5s: {}, ADE1s: {}, ADE3s: {}, ADE5s: {} '.format(
    FDE[10//cfg["model_params"]["future_step_size"]-1], FDE[30//cfg["model_params"]["future_step_size"]-1], FDE[50//cfg["model_params"]["future_step_size"]-1], np.mean(FDE[:10//cfg["model_params"]["future_step_size"]]), np.mean(FDE[:30//cfg["model_params"]["future_step_size"]]), np.mean(FDE[:50//cfg["model_params"]["future_step_size"]])))